In [ ]:
from matplotlib import pyplot as plt
from systemclass import SNAIL,SNAIL_sweep,Cavity, SNAILC, SNAILC_sweep, SNAILCC, SNAILCC_sweep
import numpy as np
from joblib import Parallel, delayed

In [ ]:
def calculate_overlap(state1, state2):
    return abs((state1.dag() * state2)[0,0])**2
def find_optimal_k(A, B, D):
    # Define a large initial minimum difference
    min_diff = float('inf')
    optimal_k = None
    
    # Iterate over a range of possible k values
    # The range can be adjusted based on expected size of k or other insights you have about your problem
    for k in range(-1000, 1000):
        # Calculate the difference for this value of k
        diff = abs(A - (B + k * D))
        
        # If this is the smallest difference we've found so far, update min_diff and optimal_k
        if diff < min_diff:
            min_diff = diff
            optimal_k = k
            
    return optimal_k

In [ ]:
from qutip import propagator,floquet_modes,Qobj,Options,basis
def energy_diff(A,omega):
    #snail parameters
    flux = 0.435
    #snail parameters
    EJ = 70*2*np.pi
    EC = 0.177*2*np.pi
    beta = 0.12
    ng = 0
    ncut = 300
    N = 3
    dims = 10
    snail = SNAIL(EJ,EC,beta,ng,ncut,flux,N,dims)

    #cavity1 parameters
    omega_c1 = 3*2*np.pi
    g_int1 = 0.15*2*np.pi
    dimc1 = 3
    cavity1 = Cavity(omega_c1,g_int1,dimc1)
    
    #cavity1 parameters
    omega_c2 = 7*2*np.pi
    g_int2 = 0.15*2*np.pi
    dimc2 = 3
    cavity2 = Cavity(omega_c2,g_int2,dimc2)
    snailcc = SNAILCC(snail,cavity1,cavity2)
    
    H0,Hc = snailcc.hamiltonian()
    energy,U = np.linalg.eigh(H0)
    U_dag = np.transpose(U.conjugate())
    Hc = U_dag @ Hc @ U
    H0 = U_dag @ H0 @ U
    H0 = H0 - H0[0][0]*np.identity(len(H0))
    index1 = np.argmin(np.abs(energy - omega_c1 * np.ones(len(energy))))
    index2 = np.argmin(np.abs(energy - omega_c2 * np.ones(len(energy))))
    bare_diff = (energy[index1] - energy[index2])/2/np.pi
    
    H0 = Qobj(H0)
    Hc = Qobj(Hc)
    args = {'w': omega}
    T = (2*np.pi)/omega
    H = [H0, [Hc, lambda t, args: A*np.cos(args['w']*t)]] 
    options = Options(nsteps=100000)  # Increasing nsteps to 10000, adjust as needed
    f_modes, f_energies = floquet_modes(H, T, args, False, options=options) 
    total_dim = dims*dimc1*dimc2
    basis_states = [basis(total_dim, index1), basis(total_dim, index2)]
    
    
    max_overlap_indices = [0,0]
    max_overlaps = [0,0]
    for f_index, f_state in enumerate(f_modes):

        # Loop over each basis state
        for b_index, b_state in enumerate(basis_states):

            # Calculate the overlap
            overlap = calculate_overlap(f_state, b_state)

            # If this is the maximum overlap for this basis state, store the index
            if overlap > max_overlaps[b_index]:
                max_overlaps[b_index] = overlap
                max_overlap_indices[b_index] = f_index
    omegac1p = f_energies[max_overlap_indices[0]]/2/np.pi
    k = find_optimal_k(omega_c1/2/np.pi, omegac1p , omega/2/np.pi)
    omegac1p = omegac1p + k *omega/2/np.pi
    omegac2p = f_energies[max_overlap_indices[1]]/2/np.pi
    k = find_optimal_k(omega_c2/2/np.pi, omegac2p, omega/2/np.pi)
    omegac2p = omegac2p + k *omega/2/np.pi
    diff = omegac2p - omegac1p
    return diff

In [ ]:
energy_diff(A,omegas[0])

In [ ]:
A = 0.5 * 2 * np.pi
omegas = np.linspace(4.011* 2 * np.pi, 4.012 * 2 * np.pi, 50)

# Parallel computation
results = Parallel(n_jobs=50)(delayed(energy_diff)(A, omega) for omega in omegas)


In [ ]:
omegas = omegas/2/np.pi

In [ ]:
plt.plot(omegas, results,'.', marker='o',)
plt.xlabel('Omega')
plt.ylabel('Energy Difference')
plt.title('Energy Difference vs Omega')
plt.show()

In [ ]:
index_of_min = np.argmin(results)
index_of_max = np.argmax(results)
detuning = (omegas[index_of_min]+omegas[index_of_max])/2

In [ ]:
import scipy
def fit_avoided_crossing(energy_diff):

    # define the cost function to fit the avoided crossing
    def cost_func(parameters):
        g = parameters[0]   # beam splitter rate 
        a = parameters[1]   # energy difference between two modes, excluding effects from avoided crossings 
        res = 0
        for i in range(len(omegas)):
            # sometimes it is helpful to select the part of data to fit with 
            # if omega_list[i]/(2*np.pi)<3.95 or omega_list[i]/(2*np.pi) > 3.96:
#             if omegas[i]<0.279:
                delta =   detuning - omegas[i]
                res += np.abs(energy_diff[i] - (a+ np.sign(delta) * np.sqrt(4*g**2 + (delta)**2) - delta ) )**2 
        return res

    # boundaries of the fitting parameters 
    bb = scipy.optimize.Bounds(np.array([0.8e-4, 4.011,]), np.array([2e-4, 4.012,]))
    
    # initial guess 
    x0 = np.array([1e-3, 4.011567357533754])
    
    res = scipy.optimize.minimize(cost_func, x0, bounds=bb, method='Powell', tol=1e-30)
    print(res)
    # store fitted beam splitter rate, unit in MHz
    g = res.x[0]   
    a = res.x[1]
    return g, a,

In [ ]:
g,a = fit_avoided_crossing(results)

In [ ]:
delta = -omegas + (detuning) *np.ones(len(omegas))
plt.plot(omegas,(a+ np.sign(delta) * np.sqrt(4*g**2 + (delta)**2) - delta ))
plt.plot(omegas,results)
# plt.ylim(0.2975,0.3)

In [ ]:
from matplotlib import pyplot as plt
from systemclass import SNAIL,SNAIL_sweep,Cavity, SNAILC, SNAILC_sweep, SNAILCC, SNAILCC_sweep
import numpy as np
from joblib import Parallel, delayed
import qutip as qt

In [ ]:
from qutip import propagator,floquet_modes,Qobj,Options,basis
def energy_diff(A,omega):
    flux = 0.435
    #snail parameters
    EJ = 70*2*np.pi
    EC = 0.177*2*np.pi
    beta = 0.12
    ng = 0
    ncut = 300
    N = 3
    dims = 10
    snail = SNAIL(EJ,EC,beta,ng,ncut,flux,N,dims)
    Hs,charge_op = snail.spectrum_charge()
    
    
    #cavity1 parameters
    omega_c1 = 3*2*np.pi
    g_int1 = 0.15*2*np.pi
    dimc1 = 3
    cavity1 = Cavity(omega_c1,g_int1,dimc1)
    
    #cavity1 parameters
    omega_c2 = 7*2*np.pi
    g_int2 = 0.15*2*np.pi
    dimc2 = 3
    cavity2 = Cavity(omega_c2,g_int2,dimc2)
    
    Hc1, Vc1 = cavity1.hamiltonian()
    Hc2, Vc2 = cavity2.hamiltonian()
    
    Ic1 = np.identity(dimc1)
    Ic2 = np.identity(dimc2)
    Is = np.identity(dims)
    
    Hs = np.kron(np.kron(Hs,Ic1),Ic2)
    Hc1 = np.kron(np.kron(Is,Hc1),Ic2)
    Hc2 = np.kron(np.kron(Is,Ic1),Hc2)
    
    H_int1 = cavity1.g_int * np.kron(np.kron(charge_op,Vc1),Ic2) * 2 * ((2 * snail.EC / snail.EJ) ** 0.25)
    H_int2 = cavity2.g_int * np.kron(np.kron(charge_op,Ic1),Vc2) * 2 * ((2 * snail.EC / snail.EJ) ** 0.25)
    
    H = Hs + Hc1 + Hc2 + H_int1 + H_int2
    Hc  =  np.kron(np.kron(charge_op,Ic1),Ic2)
    energy0,U = np.linalg.eigh(H)
#     snailcc = SNAILCC(snail,cavity1,cavity2)
#     H0,Hc = snailcc.hamiltonian()
    
#     energy,U = np.linalg.eigh(H0)

    index1 = np.argmin(np.abs(energy0 - omega_c1 * np.ones(len(energy0))))
    index2 = np.argmin(np.abs(energy0 - omega_c2 * np.ones(len(energy0))))
    detuning = energy0[index2]-energy0[index1]
    total_dim = dims*dimc1*dimc2
    H0 = Qobj(H)
    Hc = Qobj(Hc)
    psi0 = qt.basis(total_dim, index1)
    Iss = qt.qeye(dims)
    s0 = qt.basis(dims, 0)*(qt.basis(dims, 0).dag())
    cavity0 = qt.basis(dimc1, 0)*(qt.basis(dimc1, 0).dag())
    cavity1 = qt.basis(dimc1, 1)*(qt.basis(dimc1, 1).dag())
    cavity2 = qt.basis(dimc1, 2)*(qt.basis(dimc1, 2).dag())
    PP1 = qt.Qobj(np.array(qt.tensor(Iss, cavity0, cavity1)))
    PP2 = qt.Qobj(np.array(qt.tensor(Iss, cavity1, cavity0)))

    
    T = (2*np.pi)/omega
    omega1 = 4.83 * 2 * np.pi
    A1 = 0.01* 2 * np.pi
    args = {'w': omega,'w1':omega1}
    tlist = np.linspace(0, 50000 * T, 100000)  # Cover ten periods
    H = [H0, [Hc, lambda t, args: A*np.cos(args['w']*t)],[Hc, lambda t, args: A1*np.cos(args['w1']*t)]] 
#     options = Options(nsteps=100000)  # Increasing nsteps to 10000, adjust as needed
    result = qt.sesolve(H, psi0, tlist, args=args)
    exp_P1 = np.array([qt.expect(PP1, state) for state in result.states])
    exp_P2 = np.array([qt.expect(PP2, state) for state in result.states])

    return result,exp_P1,exp_P2

In [ ]:
A = 0.2 * 2 * np.pi
omega =25.204831869962916
result,exp_P1,exp_P2 = energy_diff(A,omega)

In [ ]:
# Range for omega
omega_values = np.linspace(25.204, 25.206, 50)  # Fine grid between the limits

# Parallel computation of exp_P2 for each omega
results = Parallel(n_jobs=-1)(delayed(lambda om: energy_diff(A, om)[2])(om) for om in omega_values)


In [ ]:
results = np.array(results)
omega_values = omega_values /2/np.pi

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
T= (2*np.pi)/omega
time_points = tlist = np.linspace(0, 50000 * T, 100000) 
# Create a figure and axis object
plt.figure(figsize=(12, 8))

# Plotting the heatmap, transposing the results matrix
heatmap = plt.imshow(results.T, aspect='auto', origin='lower', 
                     extent=[ omega_values.min(), omega_values.max(),time_points.min(), time_points.max()], cmap=cm.viridis)
cbar = plt.colorbar(heatmap)
cbar.set_label('Amplitude (exp_P2)')

# Labeling the axes
plt.ylabel('Time (seconds)')
plt.xlabel('Frequency (omega)')
plt.title('Amplitude (exp_P2) vs. Time and Frequency')

# Show the plot
plt.show()